In [1]:
import os
import PyPDF2 as pdf
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI  # <-- Corrected import

os.environ['OPENAI_API_KEY']= "sk-proj-6BWDTDaGs4ulJSkT9S2sT3BlbkFJYKsskXdic6VLfHsDpvFJ"

In [2]:
def getTextFromPDF()->str:
    with open('Quiz.pdf',mode='rb') as contentFile:
        convertedText = ""
        pdf_reader = pdf.PdfReader(contentFile)
        for i in range(0,len(pdf_reader.pages)):
            page = pdf_reader.pages[i]
            pageText = page.extract_text()
            convertedText+=pageText
        file_path = "Loader.txt"
        with open(file_path, "w+") as file:
            file.write(convertedText)
            file.seek(0)
        return file_path

In [3]:
loader = TextLoader(getTextFromPDF())
documents = loader.load()
text_splitter  =  RecursiveCharacterTextSplitter(chunk_size=400,chunk_overlap=50)
texts = text_splitter.split_documents(documents)
embeddings = OpenAIEmbeddings()
store = Chroma.from_documents(texts,embeddings,collection_name="Quiz-PDF")

In [4]:
llm = ChatOpenAI(temperature=0.8, model="gpt-3.5-turbo")
chain = RetrievalQA.from_chain_type(llm,retriever=store.as_retriever())

In [5]:
prompt = "The document consists of questions and multiple choice options, I want you to return a response in JSON format, the fomat contains question_number, question_text, Multiple choice questions and answer, for each question that you find in a document return a separate JSON in the provided format"
response = chain.run(prompt)

In [6]:
print(response)

```json
{
    "question_number": 1,
    "question_text": "The actual energy requirement for the separation will be many times ---------Wmin",
    "Multiple_choice_questions_and_answers": {
        "a": "greater than"
    }
}
```

```json
{
    "question_number": 2,
    "question_text": "The most commonly used to separate a solution that has a mixture of some desirable components and some that are not desirable.",
    "Multiple_choice_questions_and_answers": {
        "d": "reverse osmosis"
    }
}
```

```json
{
    "question_number": 55,
    "question_text": "Water effluent generated in printing industry is decolourised by………………….",
    "Multiple_choice_questions_and_answers": {
        "": "Electrolytic de composition"
    }
}
```

```json
{
    "question_number": 1,
    "question_text": "Major manufactured food product is ………………..(Sugar).",
    "Multiple_choice_questions_and_answers": {
        "": "Sugar"
    }
}
```

```json
{
    "question_number": 2,
    "question_text": "Qualit